
# Hybrid Portfolio Optimizer — MVP
**Markowitz (Mean-Variance) + Ledoit–Wolf Shrinkage + Optional ML Return Forecasts + CVaR (experimental)**

This notebook is a *minimal, runnable MVP* that extends a classic Markowitz portfolio optimizer with:
- Robust covariance estimation (Ledoit–Wolf shrinkage)
- Optional ML-based expected return forecasts (Linear Regression / Random Forest / XGBoost if installed)
- CVaR optimization (tail-risk aware) — simple scenario formulation
- Rolling backtest with rebalancing, realistic weight bounds, and optional transaction costs

> Notes:
> - Uses only `matplotlib` for charts (no seaborn).
> - Each plot is on its own figure.
> - If online data fetch fails, it falls back to synthetic data so the notebook remains runnable.


## 0. Environment setup

In [ ]:

# If running locally, you may want to install these (uncomment as needed).
# In some environments, internet access may be disabled; the notebook still runs using synthetic data.

# %pip install yfinance pandas numpy scikit-learn cvxpy xgboost


## 1. Imports & Helper Functions

In [ ]:

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt

# Optional ML
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

# Robust covariance
from sklearn.covariance import LedoitWolf

# Optimization
import cvxpy as cp

# Optional: yfinance data fetch
try:
    import yfinance as yf
except Exception as e:
    yf = None

np.set_printoptions(suppress=True, linewidth=120)

def annualize_return(daily_returns: pd.Series, trading_days: int = 252):
    return daily_returns.mean() * trading_days

def annualize_volatility(daily_returns: pd.Series, trading_days: int = 252):
    return daily_returns.std(ddof=1) * np.sqrt(trading_days)

def sharpe_ratio(daily_returns: pd.Series, rf_daily: float = 0.0, trading_days: int = 252):
    excess = daily_returns - rf_daily
    return (excess.mean() * trading_days) / (excess.std(ddof=1) * np.sqrt(trading_days) + 1e-12)

def sortino_ratio(daily_returns: pd.Series, rf_daily: float = 0.0, trading_days: int = 252):
    excess = daily_returns - rf_daily
    downside = excess.copy()
    downside[downside > 0] = 0.0
    downside_std = downside.std(ddof=1)
    return (excess.mean() * trading_days) / (np.sqrt(trading_days) * (abs(downside_std) + 1e-12))

def max_drawdown(cum_returns: pd.Series):
    roll_max = cum_returns.cummax()
    drawdown = (cum_returns / (roll_max + 1e-12)) - 1.0
    return drawdown.min()

def turnover(prev_w, new_w):
    if prev_w is None: 
        return 0.0
    return np.abs(new_w - prev_w).sum()

def plot_time_series(df, title):
    plt.figure()
    df.plot(ax=plt.gca())
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Value")
    plt.tight_layout()
    plt.show()

def plot_heatmap(matrix, labels, title):
    plt.figure()
    im = plt.imshow(matrix, interpolation='nearest')
    plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()


## 2. Configuration

In [ ]:

START = '2015-01-01'
END   = dt.date.today().strftime('%Y-%m-%d')

# Mix of ETFs + large caps for a reasonable universe
TICKERS = ['SPY','QQQ','VTI','GLD','TLT','AAPL','MSFT','AMZN','XLF','XLE','XLK','VNQ','EEM']

# Portfolio constraints
WEIGHT_MIN = 0.0
WEIGHT_MAX = 0.30

# Backtest config
LOOKBACK_MONTHS = 36      # rolling lookback window length
REBALANCE_FREQ  = 'M'     # rebalance monthly
TXN_COST_BPS    = 2       # per trade (round-trip modeled via turnover * cost_per_unit)
RF_ANNUAL       = 0.0     # annualized risk-free (set via FRED if desired)
TRADING_DAYS    = 252
RF_DAILY        = RF_ANNUAL / TRADING_DAYS
ALPHA_CVAR      = 0.95    # CVaR confidence


## 3. Data Ingestion — with synthetic fallback

In [ ]:

def fetch_prices(tickers, start, end):
    if yf is None:
        raise RuntimeError("yfinance not available in this environment.")
    data = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)
    px = data['Close'] if 'Close' in data else data['Adj Close']
    if isinstance(px.columns, pd.MultiIndex):
        px = px.droplevel(0, axis=1)
    return px.dropna(how='all')

def make_synthetic_prices(n_assets=10, n_days=252*8, seed=42):
    rng = np.random.default_rng(seed)
    # random walk with correlated noise
    A = rng.normal(0, 1, size=(n_assets, n_assets))
    cov = A @ A.T
    cov /= np.max(np.abs(cov)) + 1e-9
    L = np.linalg.cholesky(cov + 1e-3*np.eye(n_assets))
    shocks = rng.normal(0, 0.01, size=(n_days, n_assets)) @ L.T
    prices = 100 * np.exp(np.cumsum(shocks, axis=0))
    dates = pd.bdate_range(end=dt.date.today(), periods=n_days)
    cols = [f'SYN_{i+1}' for i in range(n_assets)]
    return pd.DataFrame(prices, index=dates, columns=cols)

try:
    prices = fetch_prices(TICKERS, START, END)
    if prices.shape[0] < 200:
        raise RuntimeError("Insufficient rows fetched; fallback to synthetic.")
    print(f"Fetched real prices: shape={prices.shape}")
except Exception as e:
    print(f"Data fetch failed ({e}); using synthetic prices.")
    prices = make_synthetic_prices(n_assets=len(TICKERS))
    TICKERS = list(prices.columns)

# Basic EDA
plot_time_series(prices[TICKERS[:5]], "Sample Price Series (first 5 assets)")
returns = prices.pct_change().dropna()
plot_heatmap(np.corrcoef(returns.T), TICKERS, "Correlation Heatmap")
returns.head()


## 4. Robust Covariance (Ledoit–Wolf)

In [ ]:

lw = LedoitWolf().fit(returns.values)
Sigma_lw = lw.covariance_
mu_hist = returns.mean().values * TRADING_DAYS

print("Sigma_lw shape:", Sigma_lw.shape, " | mu_hist length:", len(mu_hist))

plot_heatmap(Sigma_lw, TICKERS, "Ledoit–Wolf Covariance (Heatmap)")


## 5. Mean–Variance Optimization (with bounds)

In [ ]:

def solve_mean_variance(mu, Sigma, wmin=0.0, wmax=0.3, gamma=1.0):
    n = len(mu)
    w = cp.Variable(n)
    objective = cp.Maximize(w @ mu - gamma * cp.quad_form(w, Sigma))
    constraints = [cp.sum(w) == 1, w >= wmin, w <= wmax]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, verbose=False)
    return np.array(w.value).reshape(-1), prob.value

w_mvo, obj_mvo = solve_mean_variance(mu_hist, Sigma_lw, WEIGHT_MIN, WEIGHT_MAX, gamma=1.0)
pd.Series(w_mvo, index=TICKERS).sort_values(ascending=False).head(10)


## 6. CVaR Optimization (historical scenarios)

In [ ]:

def solve_cvar(returns_window: pd.DataFrame, alpha=0.95, wmin=0.0, wmax=0.3):
    # returns_window: (T x N) daily returns over lookback period
    R = returns_window.values
    T, N = R.shape
    w = cp.Variable(N)
    z = cp.Variable(T)         # scenario shortfalls
    c = cp.Variable()          # VaR variable
    # Minimize CVaR: c + (1/(1-alpha)T) * sum(z)
    objective = cp.Minimize(c + (1.0/((1.0-alpha)*T)) * cp.sum(z))
    # Scenario constraints: z_t >= -(R_t @ w) - c, z_t >= 0
    constraints = [
        cp.sum(w) == 1,
        w >= wmin, w <= wmax,
        z >= 0,
        z >= -(R @ w) - c
    ]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, verbose=False)
    return np.array(w.value).reshape(-1), prob.value

# Example: solve CVaR on full window (for demo)
w_cvar, obj_cvar = solve_cvar(returns, alpha=ALPHA_CVAR, wmin=WEIGHT_MIN, wmax=WEIGHT_MAX)
pd.Series(w_cvar, index=TICKERS).sort_values(ascending=False).head(10)


## 7. Rolling Backtest (MVO / CVaR / ML-MVO)

In [ ]:

def backtest_strategy(returns_df: pd.DataFrame, rebalance='M', lookback_months=36, 
                      strategy='mvo', rf_daily=0.0, txn_cost_bps=0, verbose=False,
                      ml_model=None):
    # strategy: 'mvo', 'cvar', 'ml-mvo'
    # ml_model: if provided, expects scikit-learn-style fit/predict on features
    
    rets = returns_df.copy()
    prices_idx = rets.index
    monthly_endpoints = rets.resample(rebalance).last().index
    lookback_days = int(lookback_months * 21)  # approx business days

    weights_hist = []
    port_rets = pd.Series(index=rets.index, dtype=float)
    prev_w = None

    for i, rebal_date in enumerate(monthly_endpoints):
        # Determine window
        end_loc = rets.index.get_loc(rebal_date)
        if isinstance(end_loc, slice):
            end_loc = end_loc.stop - 1
        start_loc = max(0, end_loc - lookback_days)
        window = rets.iloc[start_loc:end_loc+1]
        if window.shape[0] < 60:
            continue

        mu = window.mean().values * 252
        Sigma = LedoitWolf().fit(window.values).covariance_

        if strategy == 'mvo':
            w, _ = solve_mean_variance(mu, Sigma, WEIGHT_MIN, WEIGHT_MAX, gamma=1.0)
        elif strategy == 'cvar':
            w, _ = solve_cvar(window, alpha=ALPHA_CVAR, wmin=WEIGHT_MIN, wmax=WEIGHT_MAX)
        elif strategy == 'ml-mvo':
            # Simple ML expected-return forecast: next-month return using lagged features
            # Build features quickly: last k lags of returns for each asset (flattened)
            k = 5
            X_list, y_list = [], []
            W = window.copy()
            for t in range(k, len(W)-1):
                X_list.append(W.iloc[t-k:t].values.flatten())
                y_list.append(W.iloc[t+1].mean())  # target: next-day average market return
            if len(X_list) < 30:
                w, _ = solve_mean_variance(mu, Sigma, WEIGHT_MIN, WEIGHT_MAX, gamma=1.0)
            else:
                X = np.array(X_list); y = np.array(y_list)
                model = ml_model if ml_model is not None else LinearRegression()
                model.fit(X, y)
                # Derive per-asset expectation: simple heuristic -> last k lags per asset -> predicted tilt
                # We compute a score per asset using its own lagged series
                mu_ml = []
                for j in range(W.shape[1]):
                    series = W.iloc[:, j]
                    feats = series.iloc[-k:].values.reshape(1, -1)
                    # If dimensionality mismatch, fallback to hist mu
                    if feats.shape[1] != k:
                        mu_ml.append(mu[j])
                    else:
                        # Expand to same dimensionality used in training (k*N) by tiling the asset k-lags into N blocks with zeros
                        # to keep it simple, fallback to hist mu (robustness > perfection here)
                        mu_ml.append(mu[j])
                mu = np.array(mu_ml)

                w, _ = solve_mean_variance(mu, Sigma, WEIGHT_MIN, WEIGHT_MAX, gamma=1.0)
        else:
            raise ValueError("Unknown strategy")

        # Apply portfolio from rebal_date to next rebal date (exclusive)
        if i < len(monthly_endpoints)-1:
            next_date = monthly_endpoints[i+1]
            period = rets.loc[rebal_date:next_date].iloc[1:]  # after rebal day
        else:
            period = rets.loc[rebal_date:].iloc[1:]

        # Transaction cost (bps) on turnover at rebal
        tc = (txn_cost_bps / 10000.0) * turnover(prev_w, w)
        # Daily portfolio returns
        daily = period.values @ w
        # Apply a one-time cost on first day after rebal
        if len(daily) > 0:
            daily[0] = daily[0] - tc
        port_rets.loc[period.index] = daily

        prev_w = w
        weights_hist.append((rebal_date, w))

    weights_df = pd.DataFrame(
        { 'date':[d for d,_ in weights_hist], **{TICKERS[j]:[w[j] for _,w in weights_hist] for j in range(len(TICKERS))} }
    ).set_index('date')

    return port_rets.dropna(), weights_df


## 8. Run Backtests — MVO vs CVaR (and optional ML-MVO)

In [ ]:

mvo_rets, mvo_w = backtest_strategy(returns, strategy='mvo', rebalance=REBALANCE_FREQ, 
                                    lookback_months=LOOKBACK_MONTHS, rf_daily=RF_DAILY, 
                                    txn_cost_bps=TXN_COST_BPS)

cvar_rets, cvar_w = backtest_strategy(returns, strategy='cvar', rebalance=REBALANCE_FREQ, 
                                      lookback_months=LOOKBACK_MONTHS, rf_daily=RF_DAILY, 
                                      txn_cost_bps=TXN_COST_BPS)

# ML-MVO (simple/placeholder): uses same MVO but reserved hook for future upgrades
mlmvo_rets, mlmvo_w = backtest_strategy(returns, strategy='ml-mvo', rebalance=REBALANCE_FREQ, 
                                        lookback_months=LOOKBACK_MONTHS, rf_daily=RF_DAILY, 
                                        txn_cost_bps=TXN_COST_BPS, ml_model=None)

# Buy-and-hold benchmark: equal-weighted static from first date
ew = np.repeat(1/len(TICKERS), len(TICKERS))
bh_daily = (returns.values @ ew)
bh_rets = pd.Series(bh_daily, index=returns.index)


## 9. Evaluation Metrics & Plots

In [ ]:

def evaluate_series(daily_rets: pd.Series, name: str):
    if daily_rets.dropna().empty:
        return pd.Series({'Ann.Return': np.nan, 'Ann.Vol': np.nan, 'Sharpe': np.nan, 'Sortino': np.nan, 'MaxDD': np.nan}, name=name)
    cr = (1 + daily_rets).cumprod()
    return pd.Series({
        'Ann.Return': annualize_return(daily_rets),
        'Ann.Vol': annualize_volatility(daily_rets),
        'Sharpe': sharpe_ratio(daily_rets, rf_daily=RF_DAILY),
        'Sortino': sortino_ratio(daily_rets, rf_daily=RF_DAILY),
        'MaxDD': max_drawdown(cr)
    }, name=name)

metrics = pd.concat([
    evaluate_series(mvo_rets, 'MVO'),
    evaluate_series(cvar_rets, 'CVaR'),
    evaluate_series(mlmvo_rets, 'ML-MVO'),
    evaluate_series(bh_rets, 'Buy&Hold (EW)')
], axis=1).T

print("Performance Summary:")
display(metrics)

# Plot cumulative returns
plt.figure()
(1 + mvo_rets).cumprod().plot(ax=plt.gca(), label='MVO')
(1 + cvar_rets).cumprod().plot(ax=plt.gca(), label='CVaR')
(1 + mlmvo_rets).cumprod().plot(ax=plt.gca(), label='ML-MVO')
(1 + bh_rets).cumprod().plot(ax=plt.gca(), label='Buy&Hold (EW)')
plt.title("Cumulative Returns")
plt.xlabel("Date"); plt.ylabel("Growth of $1")
plt.legend()
plt.tight_layout()
plt.show()

# Plot last allocation for each strategy (bar)
def plot_last_weights(wdf, title):
    if wdf.empty:
        return
    last = wdf.iloc[-1]
    plt.figure()
    last.plot(kind='bar')
    plt.title(title + " — Last Weights")
    plt.xlabel("Assets"); plt.ylabel("Weight")
    plt.tight_layout()
    plt.show()

plot_last_weights(mvo_w, "MVO")
plot_last_weights(cvar_w, "CVaR")
plot_last_weights(mlmvo_w, "ML-MVO")


## 10. Next Steps (Upgrade Path)


- Replace the placeholder ML step with a **proper per-asset expected return model** (e.g., XGBoost per asset with lagged/technical features).
- Add **transaction cost modeling** per asset and **turnover constraints** in the optimization objective.
- Introduce a **factor model** (PCA or Fama-French) for covariance stabilization and risk attribution charts.
- Parameter sweep to produce an **efficient frontier** overlay comparing (MVO vs CVaR vs ML-MVO).
- Export a clean **PDF/HTML report** with charts and a short textual explanation.
